In [1]:
import numpy as np
import sklearn
import gradio
from keras.models import load_model

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
model = load_model('.models/mnist-cnn.h5')

C:\Users\islam\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
iface = gradio.Interface(input='sketchpad',output='class',model_obj=model, model_type='keras')
iface.launch()

Model available locally at: http://localhost:6002/
Model available publicly for 8 hours at: http://57d2bd20.ngrok.io/templates/tmp_html.html
